In [1]:
# Code modified from Konrad Hafen
# link to full article: https://towardsdatascience.com/asynchronous-parallel-programming-in-python-with-multiprocessing-a3fc882b4023

In [20]:
# Import necessary packages
import numpy as np
import time

# this package allows us to run our code in parallel
import multiprocessing as mp

# This function is defined in a script outside of this notebook. 
# For some reason, multiprocessing requires that functions used are imported from an external script
from my_function import my_function

# For completeness, I included the function here as well, but this will not redefine the function as long as redefine =False
redefine = False

if redefine == True:
    
    def my_function(i, param1, param2, param3):
        # Take the square of param1 and multiple it by param2, add param3
        result = param1 ** 2 * param2 + param3
        
        # Time sleep is used to simulate what a computationally expensive function might do.
        time.sleep(1)
        
        # return the iteration and the result
        return (i, result)

In [21]:
# Define a function that compiles the global results
def get_result(result):
    global results
    results.append(result)

In [22]:
if __name__ == '__main__':
    # Generate a 10 x 3 matrix of random numbers between 0 to 100 
    params = np.random.random((10, 3)) * 100.0
    
    # create an empty list to store results
    results = []
    
    # Record time before running the code in series
    ts = time.time()
    
    # Run through the rows of params and perform a computation with a loop (i.e run this in series)
    for i in range(0, params.shape[0]):
        get_result(my_function(i, params[i, 0], params[i, 1],params[i, 2]))
    # Print time elapses since running the code in series
    print('Time in serial:', time.time() - ts)
    print(results)
    
    # Now, let us do the same thing but in parallel
    results = []
    ts = time.time()
    # count the number of cpus present on your machine
    num_cpus = mp.cpu_count()
    print("Number of CPUs used: "+str(num_cpus))
    pool = mp.Pool(num_cpus)
    for i in range(0, params.shape[0]):
        pool.apply_async(my_function, args=(i, params[i, 0], params[i,\
         1], params[i, 2]), callback=get_result)
    pool.close()
    pool.join()
    print('Time in parallel:', time.time() - ts)
    print(results)

Time in serial: 10.044862985610962
[(0, 162703.96742876942), (1, 620013.3662997581), (2, 300767.4350129216), (3, 4165.522028034708), (4, 380600.79786331166), (5, 304912.8488240684), (6, 27214.265312522075), (7, 128863.69303980893), (8, 40263.89267852791), (9, 91824.12669183477)]
Number of CPUs used: 8
Time in parallel: 2.5763609409332275
[(5, 304912.8488240684), (4, 380600.79786331166), (6, 27214.265312522075), (3, 4165.522028034708), (1, 620013.3662997581), (7, 128863.69303980893), (2, 300767.4350129216), (0, 162703.96742876942), (8, 40263.89267852791), (9, 91824.12669183477)]
